In [1]:
import numpy as np
import pandas as pd
from tools.get_data.get_data_h5 import *
from pandasgui import show
import talib as ta
import datetime
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series
import numpy as np

import pandas as pd
pd.set_option('display.max_rows', 1000)


### 1. 获取factor数据

In [2]:
from importlib import reload
import factors as dl
reload(dl)
run = 0
if run == 1:
    data = get_data(symbol="rb99", freq="minbar", type="future",
                    source_dir="Y:/DataBase_RQ_files/")
    data["sma_diff_5"] = dl.sma_diff(data, 5)
    data["sma_diff_60"] = dl.sma_diff(data, 60)
    data["ema_diff_10"] = dl.ema_diff(data, 10)
    data["ema_diff_60"] = dl.ema_diff(data, 60)
    data["sma_of_sma_5_5"] = dl.sma_of_sma(data, 5, 5)
    data["ema_of_ema_5_5"] = dl.ema_of_ema(data, 5, 5)
    data["high_30"] = dl.high(data, 30)
    data["low_30"] = dl.low(data, 30)
    data["high_60"] = dl.high(data, 60)
    data["low_60"] = dl.low(data, 60)
    data["wl_30"] = dl.wl(data, 30)
    data["wl_60"] = dl.wl(data, 60)
    data["macd_dif"], data["macd_dea"], data["macd_hist"], data["macd_signal"] = dl.MACD(
        data, fast=10, slow=60, mid=15)
    data["slope"] = dl.slope(data, 60)
    data["rsi"] = dl.rsi(data, 60)
    data["sar"] = dl.sar(data)
    data["rwr"] = dl.rwr(data, 60)
    data["aroon_up"], data["aroon_down"], data["aroon"] = dl.aroon(data, 60)
    data["tendstrength"] = dl.tendstrength(data, 60)
    data["boll"] = dl.boll(data, 60)
    data["don"] = dl.don(data, 60)
    data["sf01"] = dl.sf01(data, 60)
    data["cor_vol"] = dl.cor_vol(data, 60)
    data["cor_oi"] = dl.cor_oi(data, 60)

    l = [1, 3, 5] + (list(range(10, 201, 5)))
    for i in l:
        data["shift_{}_rtn".format(i)] = dl.rtn_shift(data.close, i)
    print("liqka前工作完成", len(data))

    data["long_liqka"] = dl.long_liqka(data)
    data["short_liqka"] = dl.short_liqka(data)

    data.to_feather("factor.feather")
data = pd.read_feather("factor.feather")
data.dropna(inplace=True)

### 2. 因子分析

In [3]:
from importlib import reload    
import factor_analysis as fa
reload(fa)
# data =data.head(42000)

# print("factos_cols", factors_cols,"\nrtn_cols",rtn_cols)
run = 0
if run == 1:
    data.set_index("datetime", inplace=True, drop=True)
    factors_cols = []
    rtn_cols = []
    for col in data.columns:
        if col not in ['datetime', 'trading_date', "symbol"]:
            if "rtn" not in col and "liqka" not in col:
                factors_cols.append(col)
            else:
                rtn_cols.append(col)
    factors = data[factors_cols]
    rtn = data[rtn_cols]
    fal = fa.FactorAnalysis_ori()
    rank_df, results_df = fal.factor_ranked(factors, rtn,save=True,sample_size=20000)

In [ ]:
rank_df =pd.read_parquet(".//data//rank_df_mod.parquet")
results_df =pd.read_parquet(".//data//results_df_mod.parquet")
show(rank_df.head(500))


In [ ]:
factors_cols = []
rtn_cols = []
for col in rank_df.columns:
    if col not in ['datetime', 'trading_date', "symbol"]:
        if "rtn" not in col and "liqka" not in col:
            factors_cols.append(col)
        else:
            rtn_cols.append(col)


In [ ]:
from importlib import reload
import factor_analysis as fa
reload(fa)
fal = fa.FactorAnalysis_ori()
wr_df = fal.factors_select(results_df, win_rate=0,rtn=-0.1)
show(wr_df)


In [ ]:
#选择result_df中第三场index不含"liqka"的数据
show(wr_df[wr_df.index.get_level_values(2).str.contains("liqka")==False])